In [1]:
import os
import numpy as np
import cv2
from imgaug import augmenters as iaa
import imgaug as ia

## Simple Augmentations

In [2]:
sometimes = lambda aug: iaa.Sometimes(0.3, aug)

aug_list = [
    sometimes(
        iaa.OneOf([
            iaa.Crop(px=(
                0, 5 #Crop away pixels at the sides of the image.
            )), 
            iaa.CropAndPad(percent=(-0.05, 0.05)),
            #Crop or pad each side(negative values result in cropping, positive in padding).
            iaa.PiecewiseAffine(scale=(0.01, 0.03)),
            #Distort images locally by moving points around.
            iaa.Affine(
            #Affine transformations on images.
                scale={
                    "x": (0.8, 1.0),
                    "y": (0.8, 1.0)
                },
                translate_percent={
                    "x": (-0.1, 0.1),
                    "y": (-0.1, 0.1)
                },
                shear=(-8, 8),
                order=[0, 1],
                cval=255,
                mode=ia.ALL)
        ])),
    sometimes(
        iaa.OneOf([
            iaa.GaussianBlur(sigma=(0, 0.1)),
            #Blur each image with a gaussian kernel.
            iaa.AverageBlur(k=(2, 3)),
            #Blur each image using a mean over neihbourhoods.
            iaa.MedianBlur(k=(1, 3)),
            #Blur each image using a median over neihbourhoods.
            iaa.Dropout(p=(0, 0.005)),
            #Set a certain fraction of pixels in images to zero.
            iaa.Add((-5, 5)),
            #Add a value to all pixels in an image.
            iaa.ElasticTransformation(alpha=(0, 1.0), sigma=0.005),
            #Distort image locally by moving individual pixels around. 
            iaa.AdditiveGaussianNoise(scale=(0, 0.02 * 255))
            #Add gaussian noise.
        ])),
    iaa.ContrastNormalization((1.0, 2.0)),
    #Change and normalize the contrast of image.
    iaa.OneOf([
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
        #Sharpen an image, then overlay the results with the original. 
        iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
        #Emboss an image, then overlay the results with the original. 
    ]),
]

In [23]:
directory = '../data_clean/'
number_to_have = 1250

classes = [
    d for d in os.listdir(directory)
    if os.path.isdir(os.path.join(directory, d)) and not str.startswith(d, '.ipynb')
]
classes.sort()

for cl in classes:
    transformations = aug_list.copy()
    if cl in ['1', '2', '3', '5']:
        transformations.append(iaa.Fliplr(0.5))
    if cl in ['1', '2', '3']:
        transformations.append(iaa.Flipud(0.2))
    seq = iaa.Sequential(transformations, random_order=True)
    
    files = os.listdir(os.path.join(directory, cl))
    
    indicies = [int(name[:-4]) for name in files]
    indicies.sort()
    index = indicies[-1] + 1
    
    images = np.array([cv2.imread(os.path.join(directory, cl, fname)) for fname in files])
    images = files
    steps = int(np.ceil((number_to_have - len(files)) / len(files)))
    for step in range(steps):
        if step + 1 == steps:
            ind = list(range(len(files)))
            np.random.shuffle(ind)
            to_get = number_to_have - (step * len(files) + len(files))
            images = images[ind[:to_get]]
        
        images_aug = seq.augment_images(images)
        for image in images_aug:
            cv2.imwrite(os.path.join(directory, cl, str(index) + '.jpg'))
            index += 1


126
9
1251
126
9
1251
122
10
1251
124
10
1251
126
9
1251
126
9
1251
126
9
1251
125
10
1251
